# Nutrition Analysis and Recipe Finder

This project is an interactive application that allows users to look up nutritional values, find recipes, and optain cooking tips. For the menu click [here](#interactive-widget-menu)

Importing the libraries and the credentials for the api

In [ ]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import ipywidgets as widgets
import re
from IPython.display import display

# Edamam API credentials (replace with your valid keys)
edamam_api_id_nutrition = ""
edamam_api_key_nutrition = ""
edamam_api_id_recipe = ""
edamam_api_key_recipe = ""
base_url = "https://api.edamam.com/api/"


Function for API requests

In [2]:
# Function to call the Edamam API
def call_edamam_api(endpoint, params, app_id, app_key):
    try:
        full_url = f"{base_url}{endpoint}"
        params["app_id"] = app_id
        params["app_key"] = app_key
        response = requests.get(full_url, params=params, timeout=10)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}") # Print the HTTPError reason
        return None


### Overview of Features

1. [Search for Recipes](#recipe-search-function)
2. [Discover Cooking Tips](#scrape-cooking-tips-function)
3. [Get Nutritional Information](#get-nutritional-information-function)

### Recipe Search Function

In [3]:
# Function to search for recipes using Edamam Recipe API and including the changing parameters
def search_recipes(ingredient, diet=None, health=None, mealType=None):
    endpoint = "recipes/v2"
    params = {
        "q": ingredient,
        "type": "public",
    }
    if diet:
        params["diet"] = diet
    if health:
        params["health"] = health
    if mealType:
        params["mealType"] = mealType
    
    return call_edamam_api(endpoint, params, edamam_api_id_recipe, edamam_api_key_recipe)


### Scrape Cooking Tips Function

In [ ]:
# Function to scrape cooking tips
def parse_cooking_tips():
    tips_data = []
    url = "https://www.smartchicken.com/20-quick-cooking-tips"

    # Send a GET request to fetch the page content
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors
    except requests.exceptions.RequestException as e:
        print(f"Error fetching cooking tips: {e}")
        return tips_data  # Return an empty list if the request fails

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all div elements with the specified classes
    content_divs = soup.find_all('div', class_=['sqs-block html-block sqs-block-html'])

    # Extract all paragraphs
    for content_div in content_divs:
        p_tags = content_div.find_all('p')
        for p in p_tags:
            tip_text = p.get_text(strip=True)
            if tip_text:  # Ensure non-empty text is added to the tips list
                tips_data.append(tip_text)

    # Process tips to format it as a structured list of numbered tips
    formatted_tips = []
    for tip in tips_data:
        # Split the text into parts based on numbered patterns, using regex
        tips = re.split(r'(\d+\.)', tip)
        current_tip = ""
        for part in tips:
            if re.match(r'\d+\.', part):  # Check if the part is a number
                if current_tip:  # If there's already a tip being built, add it to the list
                    formatted_tips.append(current_tip.strip())
                current_tip = part  # Start a new tip with the current number
            else:
                current_tip += f" {part.strip()}"  # Adds the text to the current tip
        if current_tip:  # Add the last tip to the list
            formatted_tips.append(current_tip.strip())

    return formatted_tips


### Get Nutritional Data Function

In [5]:
# Function to get nutritional information for a specific food item
def get_nutritional_data(food_item):
    endpoint = "nutrition-data"
    params = {"ingr": food_item}
    
    return call_edamam_api(endpoint, params, edamam_api_id_nutrition, edamam_api_key_nutrition)


### Feature Selection Workflow

In [ ]:
# Function to handle feature selection
def on_feature_select(change):
    feature = change['new']
    
    if feature == "Search for Recipes":
        # Create the ingredient input
        ingredient_input = widgets.Text(placeholder="Type ingredient", description="Ingredient:")
        
        # Create the diet selection
        diet_dropdown = widgets.Dropdown(
            options=["Select Diet", "balanced", "high-fiber", "high-protein", "low-carb", "low-fat", "low-sodium"],
            description="Diet:"
        )
        
        # Create the health selection
        health_dropdown = widgets.SelectMultiple(
            options=[
                "alcohol-cocktail", "alcohol-free", "celery-free", "crustacean-free", "dairy-free", "DASH", 
                "egg-free", "fish-free", "fodmap-free", "gluten-free", "immuno-supportive", "keto-friendly", 
                "kidney-friendly", "kosher", "low-fat-abs", "low-potassium", "low-sugar", "lupine-free", 
                "Mediterranean", "mollusk-free", "mustard-free", "no-oil-added", "paleo", "peanut-free", 
                "pescatarian", "pork-free", "red-meat-free", "sesame-free", "shellfish-free", "soy-free", 
                "sugar-conscious", "sulfite-free", "tree-nut-free", "vegan", "vegetarian", "wheat-free"
            ],
            description="Health:"
        )
        
        # Create the meal type selection
        meal_type_dropdown = widgets.SelectMultiple(
            options=[
                "Breakfast", "Dinner", "Lunch", "Snack", "Teatime"
            ],
            description="Meal Type:",
        )
        
        # Search button
        def search_recipes_workflow(b):
            ingredient = ingredient_input.value
            diet = diet_dropdown.value if diet_dropdown.value != "Select Diet" else None
            health = list(health_dropdown.value)
            meal = list(meal_type_dropdown.value)
            recipes = search_recipes(ingredient, diet, health, meal)
            
            if recipes and "hits" in recipes:
                for recipe in recipes["hits"]:
                    print(f"Recipe: {recipe['recipe']['label']}\nCalories: {recipe['recipe']['calories']}\nURL: {recipe['recipe']['url']}\n")
            else:
                print("No recipes found.")
        
        search_button = widgets.Button(description="Search Recipes")
        search_button.on_click(search_recipes_workflow)
        
        display(ingredient_input, diet_dropdown, health_dropdown, meal_type_dropdown, search_button)
    
    elif feature == "Get Nutritional Information":
        # Food Item input for nutritional informaition
        food_input = widgets.Text(placeholder="Food item, with quantity (1 apple)", description="Food Item:")
        
        # Search button for nutrition data
        def get_nutrition_workflow(b):
            food_item = food_input.value
            nutrition_data = get_nutritional_data(food_item)
            
            if nutrition_data:
                print(f"Nutritional Information for {food_item}:")
                for key, value in nutrition_data.items():
                    print(f"{key}: {value}")
            else:
                print("Failed to retrieve nutritional information.")
        
        nutrition_button = widgets.Button(description="Get Nutrition Info")
        nutrition_button.on_click(get_nutrition_workflow)
        
        display(food_input, nutrition_button)
    
    # Display the cooking tips to the user
    elif __name__ == "__main__":
        tips = parse_cooking_tips()
        if tips:
            print("Cooking Tips:\n")
            for tip in tips:
                print(tip)  # Print each tip on a new line
        else:
            print("No cooking tips found.")


### Interactive widget menu

In [ ]:
# Create feature selection dropdown menu
feature_dropdown = widgets.Dropdown(
    options=["Select Feature", "Search for Recipes", "Get Nutritional Information", "Fetch Cooking Tips"],
    value="Select Feature",
    description="Feature:"
)

# Linking the selection and features
feature_dropdown.observe(on_feature_select, names='value')

# Display the dropdown menu
display(feature_dropdown)

Dropdown(description='Feature:', options=('Select Feature', 'Search for Recipes', 'Get Nutritional Information…